### Spark related

In [4]:
import findspark
findspark.init("/Users/sharmilisrinivasan/spark-2.4.3-bin-hadoop2.7/")

In [5]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession \
    .builder \
    .master("spark://ip:7077") \
    .appName("Exp") \
    .getOrCreate()

### Data

In [4]:
from pyspark.sql import Row
from pyspark.sql.functions import col

In [5]:
record = Row("time_stamp", "imsi", "enodeB", "event_id")

sample_df = spark.createDataFrame(
            [record("2017-09-09 12:00:00", "I01", "E01", "Event1"),
             record("2017-09-09 12:00:03", "I01", "E02", "Event4"),
             record("2017-09-09 12:00:06", "I01", "E02", "Event5"),
             record("2017-09-09 12:00:09", "I01", "E03", "Event6"),
             record("2017-09-09 12:00:12", "I01", "E02", "Event7"),
             record("2017-09-09 12:00:15", "I01", "E02", "Event8"),
             record("2017-09-09 12:30:00", "I01", "E01", "Event2"),
             record("2017-09-09 12:30:02", "I01", "E01", "Event3"),
             record("2017-09-09 12:30:04", "I01", "E03", "Event12"),
             record("2017-09-09 12:30:08", "I01", "E03", "Event13"),
             record("2017-09-09 12:30:10", "I01", "E04", "Event14"),
             record("2017-09-09 12:30:12", "I01", "E04", "Event15"),
             record("2017-09-09 12:30:20", "I01", "E05", "Event16")])

sample_df.show()

+-------------------+----+------+--------+
|         time_stamp|imsi|enodeB|event_id|
+-------------------+----+------+--------+
|2017-09-09 12:00:00| I01|   E01|  Event1|
|2017-09-09 12:00:03| I01|   E02|  Event4|
|2017-09-09 12:00:06| I01|   E02|  Event5|
|2017-09-09 12:00:09| I01|   E03|  Event6|
|2017-09-09 12:00:12| I01|   E02|  Event7|
|2017-09-09 12:00:15| I01|   E02|  Event8|
|2017-09-09 12:30:00| I01|   E01|  Event2|
|2017-09-09 12:30:02| I01|   E01|  Event3|
|2017-09-09 12:30:04| I01|   E03| Event12|
|2017-09-09 12:30:08| I01|   E03| Event13|
|2017-09-09 12:30:10| I01|   E04| Event14|
|2017-09-09 12:30:12| I01|   E04| Event15|
|2017-09-09 12:30:20| I01|   E05| Event16|
+-------------------+----+------+--------+



### Processing

In [6]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, collect_list, count as spark_cnt, lag, sum as spark_sum

In [7]:
window = Window.partitionBy("imsi").orderBy("time_stamp")

In [8]:
segmented_df = sample_df.\
withColumn("previ_enodeb", lag("enodeb").over(window)).\
withColumn("enodeb_change", (col("previ_enodeb" )!= col("enodeb")).cast("int")).\
fillna(0, subset=["enodeb_change"]).\
withColumn("group", spark_sum(col("enodeb_change")).over(window.rangeBetween(Window.unboundedPreceding, 0)))

In [9]:
segmented_df.show()

+-------------------+----+------+--------+------------+-------------+-----+
|         time_stamp|imsi|enodeB|event_id|previ_enodeb|enodeb_change|group|
+-------------------+----+------+--------+------------+-------------+-----+
|2017-09-09 12:00:00| I01|   E01|  Event1|        null|            0|    0|
|2017-09-09 12:00:03| I01|   E02|  Event4|         E01|            1|    1|
|2017-09-09 12:00:06| I01|   E02|  Event5|         E02|            0|    1|
|2017-09-09 12:00:09| I01|   E03|  Event6|         E02|            1|    2|
|2017-09-09 12:00:12| I01|   E02|  Event7|         E03|            1|    3|
|2017-09-09 12:00:15| I01|   E02|  Event8|         E02|            0|    3|
|2017-09-09 12:30:00| I01|   E01|  Event2|         E02|            1|    4|
|2017-09-09 12:30:02| I01|   E01|  Event3|         E01|            0|    4|
|2017-09-09 12:30:04| I01|   E03| Event12|         E01|            1|    5|
|2017-09-09 12:30:08| I01|   E03| Event13|         E03|            0|    5|
|2017-09-09 

In [10]:
segmented_df.\
groupBy(["imsi","enodeB","group"]).\
agg(collect_list("event_id"), spark_cnt("event_id")).\
drop("group").\
show()

+----+------+----------------------+---------------+
|imsi|enodeB|collect_list(event_id)|count(event_id)|
+----+------+----------------------+---------------+
| I01|   E01|              [Event1]|              1|
| I01|   E02|      [Event4, Event5]|              2|
| I01|   E03|              [Event6]|              1|
| I01|   E02|      [Event7, Event8]|              2|
| I01|   E01|      [Event2, Event3]|              2|
| I01|   E03|    [Event12, Event13]|              2|
| I01|   E04|    [Event14, Event15]|              2|
| I01|   E05|             [Event16]|              1|
+----+------+----------------------+---------------+

